# <center> Garmin Watch Data 

In [15]:
import json
import pandas as pd
import numpy as np
import os
from datetime import date, timedelta, datetime
os.chdir('/Users/jayrajparmar/Documents/side_project/health_data_tracking')
import basic_methods as bm
import time

%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
f = open('/Users/jayrajparmar/Documents/side_project/health_data_tracking/Garmin/jayrajparmar009@gmail.com_0_summarizedActivities.json', "r")
data = json.loads(f.read())

json_data = data[0]['summarizedActivitiesExport']

lis = []

for i in range(len(json_data)):
    df = pd.DataFrame([json_data[i]], columns = json_data[i].keys())
    lis.append(df)
df_garmin_exec = pd.concat(lis)


In [3]:
df_garmin_exec.head()

,activityId,uuidMsb,uuidLsb,name,activityType,userProfileId,timeZoneId,beginTimestamp,eventTypeId,rule,sportType,startTimeGmt,startTimeLocal,duration,distance,avgSpeed,avgHr,maxHr,maxRunCadence,steps,calories,bmrCalories,avgStrideLength,avgFractionalCadence,maxFractionalCadence,elapsedDuration,movingDuration,deviceId,avgDoubleCadence,maxDoubleCadence,summarizedDiveInfo,manufacturer,lapCount,waterEstimated,purposeful,autoCalcCalories,favorite,pr,elevationCorrected,decoDive,atpActivity,parent,elevationGain,elevationLoss,minElevation,maxElevation,maxVerticalSpeed,maxSpeed,startLongitude,startLatitude,vO2MaxValue,locationName,endLongitude,endLatitude,avgVerticalSpeed,floorsClimbed,floorsDescended,summarizedExerciseSets,activeSets,totalSets,totalReps,minRespirationRate,maxRespirationRate,avgRespirationRate,startStress,endStress,differenceStress
0,8959734435,-5634344091463760210,-5357954891567381561,Elliptical,elliptical,82576128,149,1654442920000,9,private,FITNESS_EQUIPMENT,"1,654,442,920,000.0000","1,654,428,520,000.0000","2,983,868.8965",0.0000,0.0000,148.0000,164.0000,79.0000,"2,258.0000","2,832.4535",318.4415,0.0000,0.1875,0.5000,"2,983,868.8965",0.0000,3317189130,48.3750,159.0000,{},GARMIN,1,298.0000,False,False,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8952433395,-1097609307350545764,-5078813204589093572,Cardio,indoor_cardio,82576128,149,1654352911000,9,private,FITNESS_EQUIPMENT,"1,654,352,911,000.0000","1,654,338,511,000.0000","144,330.9937",0.0000,0.0000,84.0000,89.0000,NaN,NaN,37.7102,12.5701,NaN,0.0000,0.0000,"144,330.9937",0.0000,3317189130,NaN,NaN,{},GARMIN,1,14.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-17,939.9994","-17,700.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8917024003,291049867221549316,-6719738246007706917,Cardio,indoor_cardio,82576128,149,1653836425000,9,private,TRAINING,"1,653,836,425,000.0000","1,653,822,025,000.0000","6,977,541.0156",0.0000,0.0000,121.0000,151.0000,NaN,NaN,"3,456.7665",745.8236,NaN,0.0000,0.0000,"6,977,541.0156",0.0000,3317189130,NaN,NaN,{},GARMIN,1,697.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-9,700.0000","-8,459.9998",0.1333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8899731934,2752421229750862679,-6627922957386941904,Cardio,indoor_cardio,82576128,149,1653606038000,9,private,TRAINING,"1,653,606,038,000.0000","1,653,591,638,000.0000","3,719,052.9785",0.0000,0.0000,153.0000,188.0000,NaN,NaN,"3,230.5054",398.0519,NaN,0.0000,0.0000,"3,719,052.9785",0.0000,3317189130,NaN,NaN,{},GARMIN,1,371.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-3,179.9999","-3,179.9999",0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,8896332721,-8383978575552557445,-4936977240004103301,Cardio,indoor_cardio,82576128,149,1653563124000,9,private,FITNESS_EQUIPMENT,"1,653,563,124,000.0000","1,653,548,724,000.0000","6,806,884.7656",0.0000,0.0000,136.0000,173.0000,NaN,NaN,"4,156.4998",729.0635,NaN,0.0000,0.0000,"6,806,884.7656",0.0000,3317189130,NaN,NaN,{},GARMIN,1,680.0000,False,False,False,False,False,False,False,False,0.0000,0.0000,"-2,100.0000","-1,400.0000",0.0100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def duration_measure(col):
#     mili_sec = int(col)
    sec = int((col/1000))
    minutes = int((sec)/60)
    return minutes


def date_range_generator(start_dt, end_dt):
    """
    Function for generating date range between start and end date;
    It has 2 arguments to be added:
        * start_dt = datetime value for the start date
        * end_dt = datetime value for the end date
    """
    lis = []
    while start_dt < end_dt:
        start_dt += timedelta(days=1)
        lis.append(start_dt)
    df = pd.DataFrame(lis, columns=['date_range'])
    df['date_range'] = pd.to_datetime(df['date_range'], errors='coerce')
    return df

In [5]:
df_garmin_imp_cols = df_garmin_exec[['activityId','name','activityType','startTimeLocal','duration',
                                       'avgHr','maxHr','calories','bmrCalories','waterEstimated']]

df_garmin_imp_cols['datetime'] = pd.to_datetime(df_garmin_imp_cols['startTimeLocal'].map(lambda x: time.strftime(
                                                        '%Y-%m-%d %H:%M:%S', time.localtime(x/1000))))

df_garmin_imp_cols['date'] = df_garmin_imp_cols['datetime'].dt.date

df_garmin_imp_cols['duration'] = df_garmin_imp_cols['duration'].astype(int)

df_garmin_imp_cols['duration_minutes'] = df_garmin_imp_cols['duration'].map(lambda x: duration_measure(x))

df_garmin_imp_cols.columns = ['activityId', 'name', 'activityType', 'startTimeLocal', 'duration',
       'avgHr', 'maxHr', 'calories', 'bmrCalories', 'waterEstimated',
       'datetime', 'date', 'duration_minutes']

In [6]:
df_garmin_imp_cols = df_garmin_imp_cols[['datetime','date','duration_minutes', 'activityType','avgHr','maxHr',
                                         'bmrCalories','calories','waterEstimated']]

df_garmin_imp_cols['date'] = pd.to_datetime(df_garmin_imp_cols['date'], errors='coerce')

In [18]:
d = {
    'duration_minutes': np.sum,
    'avgHr': np.mean,
    'maxHr': np.max,
    'bmrCalories': np.sum,
    'calories': np.sum,
    'waterEstimated': np.sum
}

df_garmin_imp_cols = df_garmin_imp_cols.groupby('date').agg(d).reset_index()

In [7]:
date_range = date_range_generator((min(df_garmin_imp_cols.date.dt.date)),(max(df_garmin_imp_cols.date.dt.date)))
final_df = date_range.merge(df_garmin_imp_cols, how='left', left_on='date_range', right_on='date')

In [ ]:
d = {
    
}

In [11]:
df_garmin_imp_cols.tail()

,datetime,date,duration_minutes,activityType,avgHr,maxHr,bmrCalories,calories,waterEstimated
0,2020-01-06 02:26:25,2020-01-06,45,indoor_cardio,138.0000,169.0000,NaN,"2,019.5896",272.0000
0,2020-01-06 02:02:42,2020-01-06,23,strength_training,128.0000,162.0000,NaN,925.9944,140.0000
0,2020-01-06 01:55:23,2020-01-06,5,strength_training,123.0000,145.0000,NaN,213.6910,31.0000
0,2020-01-05 04:32:32,2020-01-05,86,indoor_cardio,155.0000,182.0000,NaN,"4,663.4923",516.0000
0,2020-01-04 05:46:45,2020-01-04,53,indoor_cardio,154.0000,190.0000,NaN,"3,046.1445",320.0000


In [10]:
final_df.head()

,date_range,datetime,date,duration_minutes,activityType,avgHr,maxHr,bmrCalories,calories,waterEstimated
0,2020-01-05,2020-01-05 04:32:32,2020-01-05,86.0000,indoor_cardio,155.0000,182.0000,NaN,"4,663.4923",516.0000
1,2020-01-06,2020-01-06 02:26:25,2020-01-06,45.0000,indoor_cardio,138.0000,169.0000,NaN,"2,019.5896",272.0000
2,2020-01-06,2020-01-06 02:02:42,2020-01-06,23.0000,strength_training,128.0000,162.0000,NaN,925.9944,140.0000
3,2020-01-06,2020-01-06 01:55:23,2020-01-06,5.0000,strength_training,123.0000,145.0000,NaN,213.6910,31.0000
4,2020-01-07,2020-01-07 13:07:06,2020-01-07,32.0000,elliptical,140.0000,164.0000,NaN,"1,541.9274",196.0000


In [12]:
final_df[~final_df.bmrCalories.isnull()]

,date_range,datetime,date,duration_minutes,activityType,avgHr,maxHr,bmrCalories,calories,waterEstimated
585,2021-07-27,2021-07-27 15:05:47,2021-07-27,54.0000,indoor_cardio,154.0000,180.0000,351.9617,"2,966.5342",329.0000
586,2021-07-27,2021-07-27 03:02:28,2021-07-27,72.0000,indoor_cardio,134.0000,173.0000,465.0922,"2,761.2232",432.0000
588,2021-07-29,2021-07-29 15:01:47,2021-07-29,61.0000,indoor_cardio,167.0000,193.0000,398.0519,"3,548.9469",370.0000
598,2021-08-08,2021-08-08 05:42:10,2021-08-08,52.0000,indoor_cardio,155.0000,184.0000,335.2016,"2,719.3230",312.0000
599,2021-08-08,2021-08-08 05:41:46,2021-08-08,0.0000,strength_training,119.0000,122.0000,0.0000,4.1900,0.0000
600,2021-08-09,2021-08-09 15:02:04,2021-08-09,56.0000,indoor_cardio,160.0000,193.0000,364.5317,"3,100.6148",340.0000
601,2021-08-10,2021-08-10 15:03:45,2021-08-10,58.0000,indoor_cardio,157.0000,184.0000,377.1018,"3,108.9948",352.0000
603,2021-08-12,2021-08-12 15:02:58,2021-08-12,54.0000,indoor_cardio,158.0000,186.0000,347.7717,"2,849.2136",325.0000
612,2021-08-21,2021-08-21 06:47:38,2021-08-21,72.0000,indoor_cardio,152.0000,183.0000,469.2822,"3,536.3769",437.0000
620,2021-08-29,2021-08-29 07:28:13,2021-08-29,24.0000,elliptical,152.0000,174.0000,155.0307,"1,273.7661",148.0000


In [9]:
import garmin as gm



In [12]:
final_df = gm.final_df_prep()